In [1]:
%store -r trainX
%store -r testX
%store -r trainY
%store -r testY
%store -r valX
%store -r valY
%store -r le

In [8]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D,Input
from tensorflow.keras.layers import Activation, Dropout,BatchNormalization, Flatten, Dense
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
s=100
batch_size = 40
epochs = 50

In [4]:
model_base = InceptionV3(input_shape = (s, s, 3), include_top = False)

87924736/87910968 [==============================] - 84s 1us/step


In [6]:
x = Flatten()(model_base.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4, activation='sigmoid')(x)
model = keras.models.Model(model_base.input, x)
model.compile(optimizer = keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

In [7]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'auto',
    min_delta = 0,
    patience = 3,
    verbose = 1,
    restore_best_weights = True
)

In [ ]:
inc_history = model.fit(trainX,
                     trainY, 
                     epochs=epochs,
                     callbacks = [early_stopping],
                     validation_data = (valX,valY),
                     batch_size=batch_size,
                     verbose=1
)

Epoch 1/50
34/34 [==============================] - 415s 12s/step - loss: 0.5322 - acc: 0.4616 - val_loss: 0.3991 - val_acc: 0.6726
Epoch 2/50
34/34 [==============================] - 438s 13s/step - loss: 0.4112 - acc: 0.6167 - val_loss: 0.3806 - val_acc: 0.6047
Epoch 3/50
34/34 [==============================] - 440s 13s/step - loss: 0.3319 - acc: 0.7024 - val_loss: 0.2988 - val_acc: 0.7375
Epoch 4/50
34/34 [==============================] - 453s 13s/step - loss: 0.2822 - acc: 0.7644 - val_loss: 0.2292 - val_acc: 0.8142
Epoch 5/50
34/34 [==============================] - 431s 13s/step - loss: 0.2237 - acc: 0.8227 - val_loss: 0.2221 - val_acc: 0.8142
Epoch 6/50
34/34 [==============================] - 451s 13s/step - loss: 0.1811 - acc: 0.8612 - val_loss: 0.1920 - val_acc: 0.8732
Epoch 7/50
34/34 [==============================] - 478s 14s/step - loss: 0.1578 - acc: 0.8877 - val_loss: 0.1986 - val_acc: 0.8437
Epoch 8/50
27/34 [======================>.......] - ETA: 1:34 - loss: 0.1432

In [ ]:
modelLoss, modelAccuracy = model.evaluate(testX, testY, verbose=0)

print('Test Loss is {}'.format(modelLoss))
print('Test Accuracy is {}'.format(modelAccuracy ))



In [ ]:
modelLoss, modelAccuracy = model.evaluate(trainX, trainY, verbose=0)

print('Test Loss is {}'.format(modelLoss))
print('Test Accuracy is {}'.format(modelAccuracy ))


In [ ]:
# classification report on train
predIdxs = model.predict(trainX, batch_size=batch_size)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(trainY.argmax(axis=1), predIdxs, target_names=le.classes_, digits = 5))

In [ ]:
# classification report on test

predIdxs = model.predict(testX, batch_size=batch_size)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testY.argmax(axis=1), predIdxs, target_names=le.classes_, digits = 5))

In [ ]:
class_names = ['Covid-19', 'Lung Opacity', 'Normal', 'Pneumonia']
y_pred = model.predict(testX)
print(class_names[np.argmax(y_pred[11])])

In [ ]:
# Plotting Confusion Matrix
x = confusion_matrix(testY.argmax(axis=1),y_pred.argmax(axis=1))
Confusion_Matrix = pd.DataFrame(x,index=class_names, columns=class_names)

sns.set(font_scale=1.5, color_codes=True, palette='deep')
sns.heatmap(Confusion_Matrix, annot=True, annot_kws={'size':16}, fmt='d', cmap='YlGnBu')

plt.ylabel("Actual")
plt.xlabel("Predicted")
plt.title('Confusion Matrix')
plt.show()

In [ ]:
plt.plot(inc_history.history['acc'], label='accuracy')
plt.plot(inc_history.history['loss'], label = 'loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0,1])
plt.legend(loc='center right')


In [ ]:
model.save('inc_history_model.h5') 